In [12]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)
from reentry1.ballistic import run_ballistic_simulation
from reentry1.base import  ft2m, lbfsqf2Nsqm, Pa2lbfsqf, Spacecraft
from reentry1.lifting import run_lifting_simulation
from math import pi
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

plt.style.use('classic')
font = {'family' : 'monospace',
        'weight' : 'regular',
        'size'   : 13}

plt.rc('font', **font) 
plt.rc('legend',fontsize=13)

%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 10.0)
# fig.canvas.layout.width = '100%'
# fig.canvas.layout.height = '900px'

In [13]:
# from IPython.core.display import display, HTML
# display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

# Ballistic Re-entry

In [25]:

@widgets.interact_manual(betas= widgets.FloatRangeSlider(
    value=[100., 500.],
    min=30, max=6000, step=10,
    disabled=False,
    description='Betas:',
    readout_format='.1f',
    ), betastudy=widgets.Checkbox(
    value=None,
    description='Custom Beta study',
    disabled=False,
    indent=True
)
)
def ballistic_example( mission=["custom","beta_study","mercury", "mercury_parachute" ],custom_betastudy=False,  
                      input_units=["imperial", "metric"],plot_units=["imperial", "metric"], 
                      betas=500, W=2662.8, L_ref=6.2, A_ref=30.27, Cd=1.60, 
                      R_nose=1.0, V_0=22500.0, gamma_0=1.5, altitude=250000.0, orion_parachute=False,):


    if mission == "mercury_parachute":
        input_units = "imperial"
        W = 2662.8  + 2* 80 + 3*103 # lbf
        L_ref = 6.2 #ft
        A_ref = 30.27 # ft^2 
        Cd = 1.60
        R_nose = 1.0 # ft
        initial_conditions = [23_000, 1.5, 0.0, 0.0] # [V_ini [ft], gamma_ini[deg], time_ini, range_ini]
        altitude = 250_000.0 # ft
        spacecraft = Spacecraft(W, A_ref, R_nose, L_ref, Cd, Cl=None, 
                                parachute=True, imperial_units=True, beta_study=False)   # spacecraft Class stores specs and handles unit conversions
        betas = Pa2lbfsqf(spacecraft.beta)
        print("Drogue Area", ft2m(ft2m(2* (23/2.2/2)**2*pi))) # ft^2 m^2)
        print("main area" , ft2m(ft2m(3* (116/2.2/2)**2 * pi))) 

    elif mission == "mercury":
        input_units = "imperial"
        W = 2662.8  # lbf
        L_ref = 6.2 #ft
        A_ref = 30.27 # ft^2 
        Cd = 1.60
        R_nose = 1.0 # ft
        initial_conditions = [23_000, 1.5, 0.0, 0.0] # [V_ini [ft], gamma_ini[deg], time_ini, range_ini]
        altitude = 250_000.0 # ft
        spacecraft = Spacecraft(W, A_ref, R_nose, L_ref, Cd, Cl=None, 
                                parachute=False, imperial_units=True, beta_study=False)   # spacecraft Class stores specs and handles unit conversions
        betas = Pa2lbfsqf(spacecraft.beta)

    elif mission == "beta_study":
        input_units = "imperial"
        betas =  [100, 500, 1000, 5000]  # lbf/ft^2
        imperial_initial_conditions = [22500, 12.0, 0.0, 0.0] # [V_ini [ft], gamma_ini[deg], time_ini, range_ini]
        imperial_altitude = 250_000.0 # ft
        spacecraft = Spacecraft(beta_study=True, L_ref=1.0, R_nose = 1.0, imperial_units=True)

    else:
        altitude = altitude
        initial_conditions = [V_0, gamma_0, 0.0, 0.0] 
        spacecraft = Spacecraft( W=W, A_ref=A_ref, L_ref=1.0, Cd=Cd, Cl=None, parachute=orion_parachute,
                                R_nose = R_nose, imperial_units=(input_units=="imperial"))
        
        betas = np.linspace(betas[0],betas[1], num=4) if custom_betastudy is not False else Pa2lbfsqf(spacecraft.beta)

    run_ballistic_simulation(betas=betas, 
                            initial_conditions=initial_conditions,
                            altitude=altitude, spacecraft=spacecraft,
                            input_units=input_units,
                                 plot_units=plot_units)



interactive(children=(Dropdown(description='mission', options=('custom', 'beta_study', 'mercury', 'mercury_par…

# Lifting Re-entry

In [5]:
@widgets.interact_manual
def lifting_example(mission=["custom", "space_shuttle"], input_units=["imperial", "metric"], plot_units=["imperial"], W=200000.0, 
                      L_ref=107.5, A_ref=2690.0, Cd=0.84, Cl=0.84,
                      R_nose=1.0, V_0=22500.0, gamma_0s=1.5, altitude=250000.0, time_lapse=1500):

    if mission == "space_shuttle":
        input_units = "imperial"
        W = 200_000 # lbf
        A_ref = 2690 # ft^2 m^2
        L_ref = 107.5 # ft
        R_nose = 1.0  # ft
        Cd = 0.84
        Cl = 0.84
        spacecraft = Spacecraft(W, A_ref, R_nose, L_ref, Cd, Cl, 
                                    parachute=False, imperial_units=True, beta_study=False )

        beta = Pa2lbfsqf(spacecraft.beta)

        altitude = 250_000
        V_0= 23_000.0
        gamma_0s= [0.1, 1.0, 2.5]
        time_lapse = 1500 # 1300 LSODA
        
    else:
        
        spacecraft = Spacecraft( W=W, A_ref=A_ref, L_ref=1.0, Cd=Cd, Cl=None, parachute=False,
                                R_nose = R_nose, imperial_units=(input_units=="imperial"))
        beta = Pa2lbfsqf(spacecraft.beta)
     
    
    run_lifting_simulation( beta=beta,
                     V_0=V_0, gamma_0s=gamma_0s,  
                     altitude=altitude, c_L=Cl, c_D=Cd, time_elapsed=time_lapse,
                     spacecraft=spacecraft,
                     input_units=input_units, solver="RK45")





interactive(children=(Dropdown(description='mission', options=('custom', 'space_shuttle'), value='custom'), Dr…